In [78]:
import pandas as pd


In [79]:
dataset=pd.read_csv("/home/moeenuddin/Desktop/all_projects_ML/Wind_Solar_Energy_Production/data/raw/Energy Production Dataset.csv")

In [80]:
df=dataset

In [81]:
df.head()

,Date,Start_Hour,End_Hour,Source,Day_of_Year,Day_Name,Month_Name,Season,Production
0,11/30/2025,21,22,Wind,334,Sunday,November,Fall,5281
1,11/30/2025,18,19,Wind,334,Sunday,November,Fall,3824
2,11/30/2025,16,17,Wind,334,Sunday,November,Fall,3824
3,11/30/2025,23,0,Wind,334,Sunday,November,Fall,6120
4,11/30/2025,6,7,Wind,334,Sunday,November,Fall,4387


In [82]:
wind_df = df[df['Source'] == 'Wind']
solar_df = df[df['Source'] == 'Solar']


In [83]:
wind_downsampled = wind_df.sample(
    n=len(solar_df),
    random_state=42
)


In [84]:
df_balanced = pd.concat([wind_downsampled, solar_df])
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)


In [85]:
print(df_balanced['Source'].value_counts())


Source
Wind     9378
Solar    9378
Name: count, dtype: int64


In [86]:
df_balanced=df_balanced.drop(columns=["Date"])

In [87]:
df_balanced.head()

,Start_Hour,End_Hour,Source,Day_of_Year,Day_Name,Month_Name,Season,Production
0,20,21,Wind,100,Thursday,April,Spring,5375
1,3,4,Wind,285,Thursday,October,Fall,4634
2,12,13,Solar,196,Tuesday,July,Summer,4580
3,11,12,Solar,231,Saturday,August,Summer,6990
4,19,20,Wind,205,Saturday,July,Summer,4294


In [88]:
df["Day_Name"].value_counts()

Day_Name
Sunday       7416
Saturday     7416
Friday       7416
Thursday     7416
Wednesday    7416
Tuesday      7392
Monday       7392
Name: count, dtype: int64

In [89]:
# -----------------------------
# 1️⃣ Encode Source as numeric (keep all categories)
# -----------------------------
# Instead of mapping, we can one-hot encode Source too, so Wind and Solar both have explicit columns
# This way nothing is missed for ML
df_encoded = pd.get_dummies(df_balanced, 
                            columns=['Source', 'Day_Name', 'Month_Name', 'Season'],
                            drop_first=False)  # keep all categories

# -----------------------------
# 2️⃣ Convert True/False to numeric 1/0
# -----------------------------
df_encoded = df_encoded.astype(int)


In [90]:
print(df_encoded.head())
print("\nColumns after encoding:")
print(df_encoded.columns)

   Start_Hour  End_Hour  Day_of_Year  Production  Source_Solar  Source_Wind  \
0          20        21          100        5375             0            1   
1           3         4          285        4634             0            1   
2          12        13          196        4580             1            0   
3          11        12          231        6990             1            0   
4          19        20          205        4294             0            1   

   Day_Name_Friday  Day_Name_Monday  Day_Name_Saturday  Day_Name_Sunday  ...  \
0                0                0                  0                0  ...   
1                0                0                  0                0  ...   
2                0                0                  0                0  ...   
3                0                0                  1                0  ...   
4                0                0                  1                0  ...   

   Month_Name_June  Month_Name_March  Month_

In [91]:
df_encoded.head()

,Start_Hour,End_Hour,Day_of_Year,Production,Source_Solar,Source_Wind,Day_Name_Friday,Day_Name_Monday,Day_Name_Saturday,Day_Name_Sunday,...,Month_Name_June,Month_Name_March,Month_Name_May,Month_Name_November,Month_Name_October,Month_Name_September,Season_Fall,Season_Spring,Season_Summer,Season_Winter
0,20,21,100,5375,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,3,4,285,4634,0,1,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
2,12,13,196,4580,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,11,12,231,6990,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,19,20,205,4294,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [93]:
import pickle
import os

# Path to top-level artifacts folder (one level up from notebook)
save_path = "../artifacts/encoders"
os.makedirs(save_path, exist_ok=True)

# Save feature columns
feature_columns = df_encoded.drop('Production', axis=1).columns.tolist()

with open(os.path.join(save_path, "target_encoder.pkl"), "wb") as file:
    pickle.dump(feature_columns, file)

print("✅ Encoder (feature columns) saved to top-level artifacts folder!")


✅ Encoder (feature columns) saved to top-level artifacts folder!


In [95]:
import os

# Path to processed folder (one level up from notebook)
processed_path = "../data/processed"
os.makedirs(processed_path, exist_ok=True)

# Save processed dataset
df_encoded.to_csv(
    os.path.join(processed_path, "processed_energy_dataset.csv"),
    index=False
)

print("✅ Processed dataset saved to data/processed/")


✅ Processed dataset saved to data/processed/
